In [ ]:
###############
### Imports ###
###############

from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import math
from math import pi
from joblib import dump
from zipfile import ZipFile
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
#################
### Functions ###
#################

###############################
### File Manipulation Funcs ###
###############################

# File listing function
def zlist_files(data_path, zipname, cond):
    zipped_files = join(data_path, zipname)
    with ZipFile(zipped_files) as z:
        flist = [f for f in z.namelist() if cond in f]
        fout = [z.open(f) for f in flist]
        return fout

# File listing function
def list_files(data_path, cond):
    flist = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    fcond = [join(data_path, f) for f in flist if cond in f]
    return fcond

# Create pandas data loader for multi-json loads
def pandas_loader(flist, idx_name):
    dfs = [pd.read_json(f) for f in flist]
    df = pd.concat((idf.set_index(idx_name) for idf in dfs), 
                   axis=1, join='inner').reset_index()
    return df

#################################
### Feature Engineering Funcs ###
#################################

# For time based columns
def transformation(column):
    max_value = column.max()
    sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
    cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
    return sin_values, cos_values

###################
### Stats Funcs ###
###################

# Verify column skew for normalization requirement
def skewness_test(df, col_list):
    x_var = df[col_list]
    skew_data = pd.DataFrame()
    skew_data['features'] = x_var.columns
    skew_data['skew'] = x_var.skew().values
    return skew_data

# Verify Column Vif scores for multicollinearity 
def vif_test(df, col_list):
    x_var = df[col_list]
    vif_data = pd.DataFrame()
    vif_data['features'] = x_var.columns
    vif_data['vif'] = [variance_inflation_factor(x_var.values, i)\
                      for i in range(len(x_var.columns))]
    return vif_data

##################################
### Pipeline & Modelling Funcs ###
##################################
# Pipeline model with Gridsearch
def pipe_grid_model(xtrain, ytrain,
                    numeric_features, categorical_features,
                    estimator='LinearRegression'):
    
    # Numerical features pipeline with StandardScaler
    numeric_transformer = Pipeline([('scaler', StandardScaler()),
                                    ('pca', PCA())])
    # Categorical features pipeline with OHE
    categorical_transformer = OneHotEncoder(drop='first')
    
    # Combine Num and Cat feature Pipelines into preprocessor
    preprocessor = ColumnTransformer(
        transformers=[('num', numeric_transformer, numeric_features),
                     ('cat', categorical_transformer, categorical_features)])
    
    if estimator == 'LinearRegression':
        # Set pipeline with Preprocessor, Regressor
        pipe = Pipeline([('preprocessor', preprocessor),
                         ('reg', LinearRegression())])

        # Parametrs grid
        param_grid = [{'preprocessor__num__pca__n_components': np.arange(50,500,50)}]

    if estimator == 'SVR':
        # Set pipeline with Scaler, Regressor
        pipe = Pipeline([('preprocessor', preprocessor),
                         ('svr', MultiOutputRegressor(SVR()))])

        # Parametrs grid
        param_grid = [{'preprocessor__num__pca__n_components': np.arange(50,500,50),
                       'svr__estimator__kernel': ['linear', 'poly', 'rbf'],
                       'svr__estimator__degree': [2,3],
                       'svr__estimator__gamma': ['scale', 'auto'],
                       'svr__estimator__coef0': [0.0, 0.01, 0.1],
                       'svr__estimator__tol': [0.0001, 0.001, 0.01],
                       'svr__estimator__C': [0.1, 1, 3],
                       'svr__estimator__epsilon': [0.01, 0.1, 1],
                       'svr__estimator__max_iter': [100,500,1000]}]

    # Call grid function with parameters and pipeline
    grid = HalvingRandomSearchCV(pipe, param_grid, cv=10)
    grid.fit(xtrain, ytrain)
        
    # Review best params
    print(grid.best_params_)
    
    # Create model
    model = grid.best_estimator_
    
    return model

# Pipeline transform X Train and Test
def pipe_transform(xtrain, xtest,
                  numeric_features, categorical_features):
    
    # Numerical features pipeline with StandardScaler
    numeric_transformer = Pipeline([('scaler', StandardScaler())])
    # Categorical features pipeline with OHE
    categorical_transformer = OneHotEncoder(drop='first')
    
    # Combine Num and Cat feature Pipelines into preprocessor
    preprocessor = ColumnTransformer(
        transformers=[('num', numeric_transformer, numeric_features),
                     ('cat', categorical_transformer, categorical_features)])

    # Set pipeline with Scaler, Regressor
    pipe = Pipeline([('preprocessor', preprocessor)])

    # Fit transform X
    pipe.fit(xtrain)
    
    # Transform Test
    xtrainTrans = pipe.transform(xtrain)
    xtestTrans = pipe.transform(xtest)
    
    return xtrainTrans, xtestTrans

## Iteratively run model until all predictors in model are significant
## Source: https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b
# Create model using backward elimination
def backward_elimination(x, y, alpha=0.05):
    col_idx = [i for i in range(x.shape[1])]
    x = np.r_[[col_idx], x]
    removed_cols = []
    while x.shape[1]>0:
        p = []
        x1 = x[1:,:]
        # Add constant
        x1 = sm.add_constant(x1)
        # regression model
        result = sm.OLS(y, x1).fit()
        # Create list of pvalues with cols as index
        p = result.pvalues[1:].to_numpy()
        # Obtain the maximum pvalue in list
        pmax = max(p)
        # Obtain the index of the maximum pvalue
        feature_with_pmax = np.where(p == pmax)[0][0]
        # Verify if the pmax is less than 0.05 if not remove column from list
        # Run until the all values are less than 0.05
        if pmax > alpha:
            removed_cols.append(x[:1][0][feature_with_pmax])
            x = np.delete(x, feature_with_pmax, axis=1)
        else:
            break 
    return removed_cols, x1[:,1:], result

## Check for multicollinearity 
## Source: https://www.datasklr.com/ols-least-squares-regression/multicollinearity
# Remove cols with over x VIF score
def vif_elimination(x, vfactor=5):
    col_idx = [i for i in range(x.shape[1])]
    x = np.r_[[col_idx], x]
    removed_cols = []
    while x.shape[1]>0:
        v = []
        x1 = x[1:,:]
        # VIF dataframe
        # Calculating VIF for each feature
        v = [variance_inflation_factor(x1, i) for i in range(x1.shape[1])]
        # Obtain the maximum vif value in list
        vmax = max(v)
        # Obtain the index of the maximum vif value
        feature_with_vmax = np.where(v == vmax)[0][0]
        # Verify if the vmax is less than 1 if not remove column from list
        # Run until the all values are less than 1, or not correlated
        if vmax > vfactor:
            removed_cols.append(x[:1][0][feature_with_vmax])
            x = np.delete(x, feature_with_vmax, axis=1)
        else:
            break
    return removed_cols, x1

# OLS VIF Pipeline
def ols_vif_regressor(xtrain, ytrain, alphas=[0.3,0.1], vfactor=10):
    # Run OLS backwards elimination model
    xr_ols, x_ols, res = backward_elimination(xtrain, ytrain, alpha=alphas[0])
    # Run VIF removal
    xr_vif, x_vif = vif_elimination(x_ols, vfactor=vfactor)
    # Run OLS backwards elimination model again
    xr_ols2, x_ols2, res2 = backward_elimination(x_vif, ytrain, alpha=alphas[1])

    return xr_ols, xr_vif, xr_ols2, res2

# Predict scores
def predict_score(xtest, xr_ols, xr_vif, xr_ols2, res):
    # Predict Scores
    x_rc = np.delete(xtest, xr_ols, axis=1)
    x_rc1 = np.delete(x_rc, xr_vif, axis=1)
    x_rc2 = np.delete(x_rc1, xr_ols2, axis=1)
    x_rc3 = sm.add_constant(x_rc2, has_constant='add')
    pred_score = res.predict(x_rc3)
    
    return pred_score

In [ ]:
#################
### LOAD DATA ###
#################

data_path = '../data'

# Load Statistics
stats_df = pandas_loader(zlist_files(data_path, 'nfl_data_2002_2021.zip', 'stats'), 'id')
#stats_df.head(10)

# Load Record
record_df = pandas_loader(zlist_files(data_path, 'nfl_data_2002_2021.zip', 'record'), 'id')
#record_df.head(10)

# Load matches
matches_df = pd.read_json(zlist_files(data_path, 'nfl_data_2002_2021.zip', 'matches')[0])
#matches_df.head(10)

# Load teams
teams_df = pd.read_json(zlist_files(data_path, 'nfl_data_2002_2021.zip', 'teams')[0])
#teams_df.head(10)

In [ ]:
#########################
### JOIN DATA & CLEAN ###
#########################

# Remove Duplicate columns from Stats and Records df
stats_df = stats_df.loc[:,~stats_df.columns.duplicated()]
record_df = record_df.loc[:,~record_df.columns.duplicated()]

# Merge Stats and Record dfs
merged_df = stats_df.merge(record_df, how='left', on=['id'])
#merged_df.head(10)

# Merge Merged df with Teams df
remerge_df = merged_df.merge(teams_df, how='left', left_on='team_id_x', 
                             right_on='id')
#remerge_df.head(35)

# Merge Remerged df to Matches df
nfl_df = matches_df.merge(remerge_df, left_on=['season', 'home_team'], 
                          right_on=['season_x','team_id_x'])\
                   .merge(remerge_df, left_on=['season', 'away_team'], 
                          right_on=['season_x','team_id_x'],
                          suffixes=('_home', '_away'))

# Convert date_time column to datetime object
nfl_df['date_time'] = pd.to_datetime(nfl_df['date_time'], utc=True)
# Sort NFL df by date_time
nfl_df = nfl_df.sort_values(['date_time'])

# Drop redundant columns like id_x/id_y, etc
drop_cols = [col for col in nfl_df.columns if '_x_' in col or '_y_' in col]
nfl_df = nfl_df.drop(drop_cols, axis=1)

# Drop Rank and PG columns
drop_rankpg_cols = [col for col in nfl_df.columns\
                    if 'rank' in col or 'pg' in col]
nfl_df = nfl_df.drop(drop_rankpg_cols, axis=1)

# Drop stats columns with only 999 values in it
drop_999_cols = [col for col in nfl_df.columns[nfl_df.isin([999]).any()]\
                   if 'stat' in col]
nfl_df = nfl_df.drop(drop_999_cols, axis=1)

# Drop stats and record columns with only 0 values
drop_0_cols = [col for col, is_zero in ((nfl_df == 0).sum() == nfl_df.shape[0])\
               .items() if is_zero and ('stat' in col or 'record' in col)]
nfl_df = nfl_df.drop(drop_0_cols, axis=1)

# Replace 999 values in matches info
replace_999_cols = ['home_team_win', 'home_team_score', 
                    'away_team_win', 'away_team_score']
nfl_df[replace_999_cols] = nfl_df[replace_999_cols].replace(999,0)

# Drop Abbreviation team name col
drop_abb_col = [col for col in nfl_df.columns if 'abbreviation' in col]
nfl_df = nfl_df.drop(drop_abb_col, axis=1)

# Replace remaining NaNs with 0
nfl_df = nfl_df.fillna(0)

nfl_df.info()

In [ ]:
#########################
### DATA VERIFICATION ###
#########################

# Float and Int cols with Stats and Record
verification_cols = [col for col in list(nfl_df.select_dtypes\
                                         (include=['float64', 'int64']).columns)\
                    if 'stat' in col or 'record' in col]

# Skewness of the data for int and float columns
skew_data = skewness_test(nfl_df, verification_cols)
print(skew_data[(skew_data['skew'] > 0.5) | (skew_data['skew'] < -0.5)]\
      .sort_values('skew', ascending=False))
skew_data['skew'].hist(bins='auto')
plt.show()
        
# View Columns with high multicollinearity
vif_data = vif_test(nfl_df, verification_cols)
print(vif_data[vif_data['vif'] > 1].sort_values('vif', ascending=False))
vif_data[vif_data['vif'] < 1000]['vif'].hist(bins='auto')
plt.show()

In [ ]:
###########################
### FEATURE ENGINEERING ###
###########################

# Date_time to Month, Day of Week, Time of Day as ints
nfl_df['month'] = nfl_df['date_time'].dt.month.astype('int64')
nfl_df['day_week'] = nfl_df['date_time'].dt.dayofweek.astype('int64')
nfl_df['time_day'] = nfl_df['date_time'].dt.hour.astype('int64')

# Calculate Cosine and Sine for new dates
for col in ['month', 'day_week', 'time_day']:
    time_sine, time_cos = transformation(nfl_df[col])
    nfl_df[col[0:3]+'_sine'] = time_sine
    nfl_df[col[0:3]+'_cos'] = time_cos

In [ ]:
###################
### X & y SPLIT ###
###################

# X cols list
keep_cols = ['stat', 'record', 'sine', 'cos',
             'division', 'conference', 'team_name', 'location']

remove_cols = ['team_win', 'team_Score', 'id', 'date_time',
               'month', 'day_week', 'time_day']

X_cols =  [col for col in list(nfl_df.columns)\
                    if any(kc in col for kc in keep_cols)\
           and any(rc not in col for rc in remove_cols)]

## Split X and y into Train and Test
# X,y Train
X_train = nfl_df[~nfl_df['season'].isin([2021])][X_cols]
y_train = nfl_df[~nfl_df['season'].isin([2021])][['home_team_score',
                                                  'away_team_score']]
y_htrain = nfl_df[~nfl_df['season'].isin([2021])]['home_team_score']
y_atrain = nfl_df[~nfl_df['season'].isin([2021])]['away_team_score']
# X,y Test
X_test = nfl_df[nfl_df['season'].isin([2021])][X_cols]
y_test = nfl_df[~nfl_df['season'].isin([2021])][['home_team_score',
                                                 'away_team_score']]
y_htest = nfl_df[nfl_df['season'].isin([2021])]['home_team_score']
y_atest = nfl_df[nfl_df['season'].isin([2021])]['away_team_score']

In [ ]:
#######################################
### MODELLING: SM LINEAR REGRESSION ###
#######################################

# Numerical cols to StandardScaler
num_cols = [col for col in list(X_train.select_dtypes\
                                         (include=['float64', 'int64']).columns)\
                    if 'stat' in col or 'record' in col]

# Categorical cols to OHE
cat_cols = ['division_home', 'conference_home', 
            'location_home', 'team_name_home',
            'division_away', 'conference_away', 
            'location_away', 'team_name_away']

# Process and Transform X_train and X_test
X_trainTrans, X_testTrans = pipe_transform(X_train, X_test,
                                           num_cols, cat_cols)

# Run OLS Vif Pipeline on Home Games
Xr_hols, Xr_hvif, Xr_hols1, hres = ols_vif_regressor(X_trainTrans, 
                                                     y_htrain, 
                                                     alphas=[0.3,0.1], 
                                                     vfactor=10)
# Run OLS Vif Pipeline on Away Games
Xr_aols, Xr_avif, Xr_aols1, ares = ols_vif_regressor(X_trainTrans, 
                                                     y_atrain, 
                                                     alphas=[0.3,0.1], 
                                                     vfactor=10)

print(hres.summary())
print(ares.summary())

In [ ]:
###################
### SAVE MODELS ###
###################

# Model path
model_path = '../model'

# file names
home_model = 'homescore_ols_2002_2020.pickle'
away_model = 'awayscore_ols_2002_2020.pickle'

# Save files to model folder
hres.save(join(model_path, home_model))
ares.save(join(model_path, away_model))

In [ ]:
#######################################
### PREDICTIONS, TABLES, & PLOTTING ###
#######################################

## 2021 DATA 

# Predict Scores
home_score = predict_score(X_testTrans, Xr_hols, Xr_hvif, Xr_hols1, hres)
away_score = predict_score(X_testTrans, Xr_aols, Xr_avif, Xr_aols1, ares)
print(list(zip(home_score,away_score)))

In [ ]:
#######################################
### MODELLING: SK LINEAR REGRESSION ###
#######################################

# Numerical cols to StandardScaler
num_cols = [col for col in list(X_train.select_dtypes\
                                         (include=['float64', 'int64']).columns)\
                    if 'stat' in col or 'record' in col]

# Categorical cols to OHE
cat_cols = ['division_home', 'conference_home', 
            'location_home', 'team_name_home',
            'division_away', 'conference_away', 
            'location_away', 'team_name_away']

# Run Model
linreg_model = pipe_grid_model(X_train, y_train,
                               num_cols, cat_cols,
                               estimator='LinearRegression')

print(linreg_model.score(X_train, y_train))

In [ ]:
linreg_model.predict(X_test)

In [ ]:
####################################
### MODELLING: Sk SVR REGRESSION ###
####################################

# Numerical cols to StandardScaler
num_cols = [col for col in list(X_train.select_dtypes\
                                         (include=['float64', 'int64']).columns)\
                    if 'stat' in col or 'record' in col]

# Categorical cols to OHE
cat_cols = ['division_home', 'conference_home', 
            'location_home', 'team_name_home',
            'division_away', 'conference_away', 
            'location_away', 'team_name_away']

# Run Model
svr_model = pipe_grid_model(X_train, y_train,
                            num_cols, cat_cols,
                            estimator='SVR')

print(svr_model.score(X_train, y_train))

In [ ]:
###################
### SAVE MODELS ###
###################

# Model path
model_path = '../model'

# file names
svr_modelName = 'scores_svr_2002_2020.joblib'

# Save files to model folder
dump(svr_model, join(model_path, svr_modelName))

In [ ]:
svr_model.predict(X_test)